In [1]:
import pyNastran
from pyNastran.bdf.bdf import BDF

In [2]:
model = BDF()

In [3]:
from pyNastran.op2.op2 import OP2

In [4]:
modeOP2 = OP2()

In [5]:
modeOP2.read_op2('pc12-47-v1-6-static_post_m3.2_2101.op2', build_dataframe=True)

C:\ProgramData\Anaconda3\lib\site-packages\pynastran-1.2.0+dev.no.checksum.error-py3.6.egg\pyNastran\op2\op2.py:665: FutureWarning: 
Panel is deprecated and will be removed in a future version.
The recommended way to represent these types of 3-dimensional data are with a MultiIndex on a DataFrame, via the Panel.to_frame() method
Alternatively, you can use the xarray package http://xarray.pydata.org/en/stable/.
Pandas provides a `.to_xarray()` method to help automate this conversion.

  obj.build_dataframe()


In [6]:
model.read_bdf('pc12-47-v1-6-static_post_M3.2_2101.bdf',xref=True)

In [7]:
# f = open('junk.out', 'w')

In [8]:
# >>> for pid, prop in sorted(model.properties.items()):
# >>>     f.write(prop.write_card(size=8, is_double=False))

In [9]:
# >>> for eid,element in sorted(model.elements.items()):
# >>>     print('eid=%s type=%s\n' %(eid, element.type))

In [10]:
# prop = model.properties[7300029]

In [11]:
pid_to_eids_map = model.get_property_id_to_element_ids_map()

In [12]:
# pid_to_eids_map[7300029]

In [13]:
# print(model.get_bdf_stats())

In [14]:
import pandas as pd
import numpy as np

In [15]:
# # pid_df  = pd.DataFrame.from_dict(pid_to_eids_map)

# pid_df  = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in pid_to_eids_map.items() ]))
# pid_df.head()

In [16]:
# pid_df = pid_df.fillna(0)
# count = pid_df.apply(np.count_nonzero, axis=0)

In [17]:
# count.sort_values()

In [18]:
# prop = model.properties[7300029]

In [19]:
# prop.Area()

In [20]:
# df = pd.DataFrame(list(pid_to_eids_map.keys()))
# df = df.rename(columns={df.columns[0]:'PropID'})
# df.head()

In [21]:
# df['PropType'] = df.PropID.map(lambda x: model.properties[x].type)

In [22]:
# df_pbar = df[df.PropType == 'PBAR']

In [23]:
# df_pbar.head()

In [24]:
# pbar = model.properties[1110301]

In [25]:
# model.properties[1110301].Area()

In [26]:
# pbar

In [27]:
# pbar.Area()

In [28]:
# df_pbeam = df[df.PropType == 'PBEAM']

In [29]:
# df_pbeam.head()

In [30]:
# pbeam = model.properties[1416002]
# pbeam

In [31]:
# pbeam.Area()

In [32]:
def ReadOP2Files(path):
    path = path + '*.op2'
    print('Given File path is : \n' , path , '\n')
    op2ListName = glob.glob(path)
    print('Found the following list of OP2 Files: \n')
    print('========================================', '\n')
    op2ListName
    dictOP2FileList = {}
    for i in range(len(op2ListName)):
        #print(colors[i])
        dictOP2FileList[os.path.splitext(os.path.basename(op2ListName[i]))[0]] = op2ListName[i]
        #print(os.path.basename(op2ListName[i]), op2ListName[i])
        #print(os.path.splitext(os.path.basename(op2ListName[i])))
    return dictOP2FileList


# In[46]:


def ReadOP2File(op2FileName):
    preOP2Model = OP2()
    preOP2File = op2FileName
    # Load Pre OP2 File
    preOP2Model.read_op2(preOP2File, build_dataframe=True)
    return preOP2Model


# In[47]:


def ReadLoadcases(preOP2Model):
    print(preOP2Model.get_op2_stats(short=True))
    preLCs = list(preOP2Model.cshear_stress.keys())
    return preLCs


# In[48]:


# Function Which creates new index for Shear Stress individual loadcase
def newShearIndex(lc):
    colReInx = pd.MultiIndex.from_tuples([(lc, 'max_shear'), (lc, 'avg_shear'), (lc, 'margin')])
    return colReInx


# In[49]:


def ReadShearStressResults(preOP2Model):
    preLCs = ReadLoadcases(preOP2Model)
    # Start with an empty dataframe [note: Logic to be changed later]
    cShearResultDF_Pre = pd.DataFrame({'A' : []})
    for lc in preLCs:
        tempDF = preOP2Model.cshear_stress[lc].data_frame
        tempDF = tempDF.unstack(level=0).T
        tempDF1 = tempDF.reindex(columns=newShearIndex(lc), level=1)
        if cShearResultDF_Pre.empty == False:
            cShearResultDF_Pre = pd.concat([cShearResultDF_Pre, tempDF1], axis=1,sort=False)
        else :
            cShearResultDF_Pre = tempDF1.copy()
    return cShearResultDF_Pre

In [33]:
# df_pbeam = df[df.PropType == 'PBEAM']
# df_pbeam.head()

In [44]:
beam_stresses_df = modeOP2.cbeam_force[92101].data_frame
# modeOP2.cbeam_force.keys()
beam_stresses_df['element_type'] = beam_stresses_df.ElementID.map(lambda x: model.elements[x].type)
beam_stresses_df['property_id'] = beam_stresses_df.ElementID.map(lambda x: model.elements[x].pid)
beam_stresses_df['element_area'] = beam_stresses_df.property_id.map(lambda x: model.properties[x].Area())
beam_stresses_df['beam_stress'] = beam_stresses_df.axial_force / beam_stresses_df.element_area
interested_columns  =['ElementID', 'Location', 'sd', 'axial_force', 'element_type', 'property_id', 'element_area', 'beam_stress']
beam_stresses_df = beam_stresses_df[interested_columns]
beam_stresses_df.head()

,ElementID,Location,sd,axial_force,element_type,property_id,element_area,beam_stress
0,1117503.0,0.0,0.0,-409.992432,CBEAM,7300029,370.135,-1.107683
1,1117503.0,1.0,1.0,-409.992432,CBEAM,7300029,370.135,-1.107683
2,1117507.0,0.0,0.0,1002.093323,CBEAM,5500314,370.135,2.707373
3,1117507.0,1.0,1.0,1002.093323,CBEAM,5500314,370.135,2.707373
4,1118503.0,0.0,0.0,1896.697876,CBEAM,7300029,370.135,5.124341


In [47]:
pbeam_stress = modeOP2.cbeam_stress[92101].data_frame

In [52]:
pbeam_stress.head(10)

Static                                       0
ElementID NodeID  Item                        
1117503   1117017 sxc            -1.265965e+00
                  sxd            -1.265965e+00
                  sxe            -1.265965e+00
                  sxf            -1.265965e+00
                  smax           -1.265965e+00
                  smin           -1.265965e+00
                  MS_tension      1.401298e-45
                  MS_compression  1.401298e-45
          7999087 sxc            -3.191498e-01
                  sxd            -3.191498e-01

In [42]:
beam_stresses_df.set_index('ElementID', inplace=True)
beam_stresses_df.head()

,Location,sd,axial_force,element_type,property_id,element_area,beam_stress
ElementID,,,,,,,
1117503.0,0.0,0.0,-409.992432,CBEAM,7300029,370.135,-1.107683
1117503.0,1.0,1.0,-409.992432,CBEAM,7300029,370.135,-1.107683
1117507.0,0.0,0.0,1002.093323,CBEAM,5500314,370.135,2.707373
1117507.0,1.0,1.0,1002.093323,CBEAM,5500314,370.135,2.707373
1118503.0,0.0,0.0,1896.697876,CBEAM,7300029,370.135,5.124341


In [ ]:
bar_Stress_calc = modeOP2.cbar_force[92101].data_frame
bar_Stress_calc = bar_Stress_calc.unstack()
# modeOP2.cbeam_force.keys()
bar_Stress_calc['ElementType'] = bar_Stress_calc.ElementID.map(lambda x: model.elements[x].type)
bar_Stress_calc['PropID'] = bar_Stress_calc.ElementID.map(lambda x: model.elements[x].pid)
bar_Stress_calc['ElementArea'] = bar_Stress_calc.PropID.map(lambda x: model.properties[x].Area())
# bar_Stress_calc['beam_stress'] = bar_Stress_calc.axial_force / bar_Stress_calc.ElementArea
bar_Stress_calc.head()

In [ ]:
bar_Stress_calc = modeOP2.cbar_force[92101].data_frame

In [ ]:
bar_Stress_calc = bar_Stress_calc.unstack()

In [ ]:
bar_Stress_calc['ElementType'] = bar_Stress_calc.ElementID.map(lambda x: model.elements[x].type)

In [ ]:
bar_Stress_calc = bar_Stress_calc.reset_index()
bar_Stress_calc.head()

In [ ]:
bar_Stress_calc.columns = bar_Stress_calc.columns.get_level_values(

In [ ]:
bar_Stress_calc.head()

In [ ]:
bar_Stress_calc.columns.names

In [ ]:
bar_Stress_calc.columns.names

In [ ]:
df.bar_Stress_calc(columns={'oldName1': 'newName1', 'oldName2': 'newName2'}, inplace=True)